# Run inference on time to merge model trained previously


## What we did previously

In the previous [notebook](./03_model_training.ipynb) we trained machine learning models to classify a PR's `time_to_merge` into one of the 10 bins (or "classes"). We then deployed the model with the highest f1-score as a service using the model saved in s3.

## In this step

In this notebook, we are going to fetch the model that we previously trained, saved and stored in s3. We will send a payload to this model and see how it performs on the test data.
# Time to Merge Prediction Inference Service

In the previous notebook, we explored some basic machine learning models for predicting time to merge of a PR.

In [1]:
import os
import sys
import gzip
import json
import boto3
import datetime
import requests
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

import joblib

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

load_dotenv(find_dotenv(), override=True)

import warnings
warnings.filterwarnings('ignore') 

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
class CephCommunication:
    """
    Class to establish communication with a ceph s3 bucket.
    It connects with the bucket and provides methods to read and write data in the parquet format.
    """

    def __init__(
        self, s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
    ):
        self.s3_endpoint_url = s3_endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.s3_resource = boto3.resource(
            "s3",
            endpoint_url=self.s3_endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
        )
        self.bucket = s3_bucket
        ## Todo: Add try catch

    def upload_to_ceph(self, dataframe, s3_path, filename):
        """
        This helper function takes as input the data frame to be uploaded, and the output filename.
        It then saves the data frame in the defined ceph bucket.
        """
        parquet_buffer = BytesIO()
        dataframe.to_parquet(parquet_buffer)
        s3_obj = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        status = s3_obj.put(Body=parquet_buffer.getvalue())
        return status

    def read_from_ceph(self, s3_path, filename):
        """
        Helper function to read from ceph and see if the saved data is correct.
        """
        buffer = BytesIO()
        s3_object = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        s3_object.download_fileobj(buffer)
        df_temp = pd.read_parquet(buffer)
        return df_temp


def save_to_disk(dataframe, path, filename):
    """
    Helper function to save the dataframe
    as a parquet file to disk.
    """
    dataset_base_path = Path(path)
    dataset_base_path.mkdir(parents=True, exist_ok=True)
    dataframe.to_parquet(f"{path}/{filename}")
    return True

In [3]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,

ORG = os.getenv("GITHUB_ORG")
REPO = os.getenv("GITHUB_REPO")

## S3 bucket credentials
s3_endpoint_url = os.getenv("S3_ENDPOINT_URL")
s3_access_key = os.getenv("AWS_ACCESS_KEY_ID")
s3_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_bucket = os.getenv("S3_BUCKET")

s3_input_data_path = os.getenv("CEPH_BUCKET_PREFIX")

REMOTE = os.getenv("REMOTE")
RAW_DATA_PATH = os.path.join(
    s3_input_data_path, "srcopsmetrics/bot_knowledge", ORG, REPO, "PullRequest.json"
)

In [4]:
output = []
local_input_data_path = "../../../data/raw/GitHub/PullRequest.json.gz"
if REMOTE:
    print("getting dataset from ceph")
    s3 = boto3.resource(
        "s3",
        endpoint_url=s3_endpoint_url,
        aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key,
    )
    content = s3.Object(s3_bucket, RAW_DATA_PATH)
    file = content.get()["Body"].read().decode("utf-8")

    prs = json.loads(file)

    for pr in prs.splitlines():
        output.append(json.loads(pr))

else:
    print("getting dataset from local")
    with gzip.open(local_input_data_path, "r") as f:
        prs = json.loads(f.read().decode("utf-8"))


pr_df = pd.DataFrame(output)

getting dataset from ceph


In [5]:
# github pr dataset collected using thoth's mi-scheduler
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Release of version 2.12.0,"Hey, @xtuchyna!\n\nOpening this PR to create a...",M,khebhut[bot],1665600542,1.665601e+09,sesheta,1.665601e+09,sesheta,1,2,{'sesheta': 327},"{'1139569371': {'author': 'sefkhet-abwy[bot]',...","[approved, bot, size/M, ok-to-test]",[2eadea6d1417c3baede8d3b32ea1cd01c37bb319],"[CHANGELOG.md, srcopsmetrics/__init__.py]",1.665601e+09,1.665601e+09,612
1,Automatic update of dependencies by Kebechet f...,Kebechet has updated the dependencies to the l...,XL,khebhut[bot],1665600524,NaN,None,NaN,None,1,1,"{'sesheta': 187, 'khebhut[bot]': 6}","{'1139569050': {'author': 'sefkhet-abwy[bot]',...","[approved, bot, size/XL, ok-to-test]",[fcd36e9e238c243c73ca0a214ed373b2cf5c1c35],[Pipfile.lock],1.665601e+09,1.665601e+09,611
2,Mi fixes,## Related Issues and Dependencies\r\nrelated ...,L,Gkrumbach07,1663711269,1.665600e+09,sesheta,1.665600e+09,sesheta,8,6,"{'xtuchyna': 13, 'Gkrumbach07': 24, 'sesheta':...","{'1118375791': {'author': 'xtuchyna', 'words_c...","[approved, size/L]","[29f4ffd578ee20ff379765d7835dc5a306832a3c, 98e...","[srcopsmetrics/cli.py, srcopsmetrics/entities/...",1.663938e+09,1.665600e+09,609
3,Automatic update of base-image in CI,Automatic update of base-image in CI.,XS,sesheta,1663342324,1.663345e+09,sesheta,1.663345e+09,sesheta,1,1,{'sesheta': 140},"{'1110874496': {'author': 'sefkhet-abwy[bot]',...","[approved, size/XS, ok-to-test]",[d5d590ff2a244cb4d2baee2682ee9b982b4b5c19],[.aicoe-ci.yaml],1.663342e+09,1.663342e+09,608
4,Automatic update of dependencies by Kebechet f...,Kebechet has updated the dependencies to the l...,L,khebhut[bot],1663342276,1.663344e+09,sesheta,1.663344e+09,sesheta,1,1,{'sesheta': 397},"{'1110873483': {'author': 'sefkhet-abwy[bot]',...","[approved, size/L, bot, ok-to-test]",[a676faae110a141fb409c6d7cd959d364a8cfe50],[Pipfile.lock],1.663342e+09,1.663342e+09,607


In [6]:
interval = (pr_df["merged_at"] - pr_df["created_at"]).astype("float")
interval = interval.dropna()
interval

0          756.0
2      1888847.0
3         2731.0
4         1340.0
5          871.0
         ...    
410    1890777.0
411    1177800.0
412     488406.0
413    2159397.0
415        382.0
Length: 384, dtype: float64

In [7]:
n_buckets = 10

quantiles = interval.quantile(q=np.arange(0, 1 + 1e-100, 1 / n_buckets))
quantiles

0.0         7.0
0.1        37.3
0.2       118.6
0.3       136.0
0.4       260.2
0.5       607.5
0.6      1228.6
0.7      5008.4
0.8     71007.0
0.9    434048.3
dtype: float64

In [8]:
quantiles / 3600

0.0      0.001944
0.1      0.010361
0.2      0.032944
0.3      0.037778
0.4      0.072278
0.5      0.168750
0.6      0.341278
0.7      1.391222
0.8     19.724167
0.9    120.568972
dtype: float64

In [9]:
time_intervals = quantiles / 3600

In [10]:
# remove PRs from train/test which are still open
pr_df = pr_df[pr_df["closed_at"].notna()]
pr_df = pr_df[pr_df["merged_at"].notna()]

In [11]:
pr_df["created_at"] = pr_df["created_at"].apply(
    lambda x: int(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["closed_at"] = pr_df["closed_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["merged_at"] = pr_df["merged_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)

In [12]:
TEST_DATA_PATH = os.path.join(s3_input_data_path, ORG, REPO, "test-data")

# read processed and split data created for train/test in the model training notebook
if REMOTE:
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    X_test = cc.read_from_ceph(TEST_DATA_PATH, "X_test.parquet")
    y_test = cc.read_from_ceph(TEST_DATA_PATH, "y_test.parquet")

else:
    print(
        "The X_test.parquet and y_test.parquet files are not included in the github repo."
    )
    print(
        "Please set REMOTE=1 in the .env file and read this data from the S3 bucket instead."
    )

In [13]:
X_test

,size,created_at_day,created_at_month,created_at_weekday,created_at_hour,changed_files_number,body_size,commits_number,filetype_.py,filetype_.lock,...,title_wordcount_var,title_wordcount_version,title_wordcount_visualization,title_wordcount_visualizations,title_wordcount_waiting,title_wordcount_wip,title_wordcount_workflows,title_wordcount_xtuchyna,title_wordcount_yaml,title_wordcount_⬆️
288,2.0,21.0,6.0,6.0,21.0,1.0,14.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269,3.0,21.0,8.0,4.0,10.0,1.0,14.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
379,3.0,10.0,2.0,0.0,20.0,1.0,14.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
355,3.0,25.0,3.0,2.0,23.0,1.0,14.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,4.0,28.0,1.0,4.0,13.0,2.0,2555.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,3.0,23.0,6.0,2.0,10.0,1.0,59.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181,1.0,4.0,12.0,4.0,10.0,1.0,35.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,1.0,19.0,11.0,4.0,3.0,2.0,76.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,0.0,24.0,9.0,3.0,12.0,1.0,14.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_test

,ttm_class
288,3
269,0
379,3
355,3
60,4
...,...
114,9
181,7
69,5
252,1


In [15]:
sample_payload = pr_df.reindex(X_test.index)

In [16]:
sample_payload

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
288,Automatic update of dependency numpy from 1.18...,"Dependency numpy was used in version 1.18.5, b...",M,sesheta,1,1.592775,None,1.592775,None,1,1,{'ghost': 36},{},"[approved, bot]",[36d4439cc7470e9c4abb845b4f23c5b308f5faf9],[Pipfile.lock],NaN,NaN,162
269,Automatic update of dependency pandas from 1.1...,"Dependency pandas was used in version 1.1.0, b...",L,sesheta,1,1.598004,sesheta,1.598004,sesheta,1,1,{'sesheta': 140},"{'472364172': {'author': 'sefkhet-abwy[bot]', ...","[approved, size/L, bot]",[eb100a241e6d264df8dc8942511e9a8003bb6898],[Pipfile.lock],1.598004e+09,1.598004e+09,193
379,Automatic update of dependency thoth-storages ...,Dependency thoth-storages was used in version ...,L,sesheta,1,1.581366,None,1.581366,None,1,1,{'ghost': 18},{},"[size/L, bot]",[f58acfb70779bdd15ff2a25cbeae1494e9983c45],[Pipfile.lock],NaN,NaN,45
355,Automatic update of dependency numpy from 1.18...,"Dependency numpy was used in version 1.18.1, b...",L,sesheta,1,1.585179,None,1.585179,None,1,1,{'ghost': 18},{},"[size/L, bot]",[64e5824b94dd52bd61b16147ffdda93e99f25192],[Pipfile.lock],NaN,NaN,72
60,Release of version 2.10.2,"Hey, @xtuchyna!\n\nOpening this PR to fix the ...",XL,khebhut[bot],1,1.643376,sesheta,1.643376,sesheta,1,2,{'sesheta': 187},"{'866125820': {'author': 'sefkhet-abwy[bot]', ...","[approved, bot, size/XL, needs-ok-to-test, ok-...",[8f493889933cff6c5bdbda2274b6855d5e3accb7],"[CHANGELOG.md, srcopsmetrics/__init__.py]",1.643376e+09,1.643376e+09,508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Automatic update of dependencies by Kebechet,Kebechet has updated the depedencies to the la...,L,khebhut[bot],1,1.625578,sesheta,1.625578,sesheta,1,1,"{'sesheta': 185, 'khebhut[bot]': 70, 'xtuchyna...","{'690480664': {'author': 'sefkhet-abwy[bot]', ...","[approved, size/L, bot, ok-to-test]",[7a1cf3abb014c13a1bfb213b5f5fce3bdb4e4b5c],[Pipfile.lock],1.624444e+09,1.624444e+09,421
181,Multiple entities now passed as comma sep. string,## Related Issues and Dependencies\r\nhttps://...,S,xtuchyna,1,1.607090,sesheta,1.607090,sesheta,1,1,{'sesheta': 65},"{'544977924': {'author': 'pacospace', 'words_c...","[approved, size/S]",[cb346df338364e1593b5a588742ee401a45dbe5d],[srcopsmetrics/cli.py],1.607090e+09,1.607090e+09,296
69,Release of version 2.10.1,"Hey, @xtuchyna!\n\nOpening this PR to fix the ...",S,khebhut[bot],1,1.637292,sesheta,1.637292,sesheta,1,2,{'sesheta': 327},"{'810723762': {'author': 'sefkhet-abwy[bot]', ...","[approved, bot, size/S, needs-ok-to-test, ok-t...",[de5e542b13768d79b9106f84e14a622fd199c635],"[CHANGELOG.md, srcopsmetrics/__init__.py]",1.637292e+09,1.637292e+09,495
252,Automatic update of dependency dash from 1.14....,"Dependency dash was used in version 1.14.0, bu...",XS,sesheta,1,1.600951,sesheta,1.600951,sesheta,1,1,{'sesheta': 140},"{'495533323': {'author': 'sefkhet-abwy[bot]', ...","[approved, size/XS, bot]",[1f7f7b37fa43c753ee5f47cbc3181bbb97fe8da5],[Pipfile.lock],1.600952e+09,1.600952e+09,213


In [17]:
sample_payload.dtypes

title                    object
body                     object
size                     object
created_by               object
created_at                int64
closed_at               float64
closed_by                object
merged_at               float64
merged_by                object
commits_number            int64
changed_files_number      int64
interactions             object
reviews                  object
labels                   object
commits                  object
changed_files            object
first_review_at         float64
first_approve_at        float64
id                       object
dtype: object

In [18]:
## read model
MODEL_KEY = os.path.join(s3_input_data_path, ORG, REPO, "ttm-model")
MODEL_FILENAME = "model.joblib"


s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

buffer = BytesIO()
s3_object = s3_resource.Object(s3_bucket, f"{MODEL_KEY}/{MODEL_FILENAME}")
s3_object.download_fileobj(buffer)
model = joblib.load(buffer)
model

Pipeline(steps=[('extract_features',
                 ColumnTransformer(transformers=[('pr_size',
                                                  OrdinalEncoder(categories=[['XS',
                                                                              'S',
                                                                              'M',
                                                                              'L',
                                                                              'XL',
                                                                              'XXL']]),
                                                  ['size']),
                                                 ('created_at_details',
                                                  DateTimeDetailsTransformer(),
                                                  ['created_at']),
                                                 ('n_changed_files',
                                                  NumChangedFilesTransformer(),
                                                  ['changed_files_number']),
                                                 ('body_size',
                                                  StringLenTransformer(),
                                                  ['body']),
                                                 ('n_commits', 'p...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.1, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=125, n_jobs=-1, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=42, reg_alpha=0, ...))])

In [19]:
# Test model on the dataset
preds = model.predict(sample_payload)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00         6
           2       0.30      0.86      0.44         7
           3       0.33      0.17      0.22        12
           4       0.00      0.00      0.00        13
           5       0.00      0.00      0.00        11
           6       0.21      0.43      0.29         7
           7       0.12      0.50      0.19         6
           8       0.00      0.00      0.00         3
           9       1.00      0.40      0.57         5

    accuracy                           0.21        77
   macro avg       0.20      0.24      0.17        77
weighted avg       0.17      0.21      0.15        77



# Conclusion

This notebook fetches the saved model from s3 and sends a payload to see how the model is performing on this new data. Additionally, we see that the evaluation scores in the classification report match the ones we saw in the training notebook. So, great, looks like our model are working as expected, and are ready to predict some times to merge for GitHub PRs! 